In [5]:
from pathlib import Path

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler

In [27]:

# load data
df_train_full = pd.read_csv("traintitanic.csv")
df_test = pd.read_csv("test.csv")

#show data
# show data for training
df_train_full

#after see the data we can get that on 

PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ...    ...   
886                              Montvila, Rev. Juozas    male  27.0      0   
887                       Graham, Miss. Margaret Edith  female  19.0      0   
888           Johnston, Miss. Catherine Helen "Carrie"  female   NaN      1   
889                              Behr, Mr. Karl Howell    male  26.0      0   
890                                Dooley, Mr. Patrick    male  32.0      0   

     Parch            Ticket     Fare Cabin Embarked  
0        0         A/5 21171   7.2500   NaN        S  
1        0          PC 17599  71.2833   C85        C  
2        0  STON/O2. 3101282   7.9250   NaN        S  
3        0            113803  53.1000  C123        S  
4        0            373450   8.0500   NaN        S  
..     ...               ...      ...   ...      ...  
886      0            211536  13.0000   NaN        S  
887      0            112053  30.0000   B42        S  
888      2        W./C. 6607  23.4500   NaN        S  
889      0            111369  30.0000  C148        C  
890      0            370376   7.7500   NaN        Q  

[891 rows x 12 columns]

In [ ]:
# show data for testing this is a forbid because in fact we can do and predict exactly on futures events 
# you can know the problems of test before it is pulish
# but in this time i want to show you see the difference of Train and Test data. and what thing will be went do it
df_test

In [13]:
# Make clean data it mean delete columns which not support for prediction
df_train_full.drop(columns=["Cabin"])
df_test.drop(columns=["Cabin"]);

In [14]:
#Create Pipeline i can read it into Machine Learning was written by me search validation it is a method was apply it there
#After that i try set label for data
df_train, df_val = train_test_split(df_train_full, test_size=0.1)
X_train = df_train.copy()
y_train = X_train.pop("Survived")

X_val = df_val.copy()
y_val = X_val.pop("Survived")

In [15]:
cat_cols = ["Embarked", "Sex", "Pclass"]
cat_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse=False)),
    ]
)

In [16]:
num_cols = ["Age", "Fare"]
num_transformer = Pipeline(
    steps=[("imputer", KNNImputer(n_neighbors=5)), ("scaler", RobustScaler())]
)

In [17]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_cols),
        ("cat", cat_transformer, cat_cols),
    ]
)

In [18]:
# Full training pipeline
full_pp = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier())]
)

# training
full_pp.fit(X_train, y_train)

# training metric
y_train_pred = full_pp.predict(X_train)
print(f"Accuracy score on train data: {accuracy_score(list(y_train), list(y_train_pred)):.2f}")

# validation metric
y_pred = full_pp.predict(X_val)
print(f"Accuracy score on validation data: {accuracy_score(list(y_val), list(y_pred)):.2f}")

Accuracy score on train data: 0.98
Accuracy score on validation data: 0.84


In [19]:
# make submission
preds = full_pp.predict(df_test)
sample_submission = pd.read_csv("gender_submission.csv")
sample_submission["Survived"] = preds
sample_submission.to_csv("titanic_submission.csv", index=False)

In [21]:
%%capture
!kaggle competitions submit -c titanic -f titanic_submission.csv -m "simple submission"

'rm' is not recognized as an internal or external command,
operable program or batch file.
